### RFM 고객 분석
1. 식료품 구매하는 소비자들을 분석 목적
2. 오프라인과 온라인을 나눠서 분석하고 어떤 차이점이 있는지 확인하기
3. 추가적으로 오프라인에서도 대형마트와 일만 마트가 구분되기 때문에 이에 대한 분석도 가능할 듯

참고문헌: https://zephyrus1111.tistory.com/13

In [ ]:
import pandas as pd

card_filter_data = pd.read_csv("Desktop/card_data_filter.csv")

In [ ]:
# online과 offline으로 분류하기
offline = card_filter_data[card_filter_data['large_category'] == '생활/마트']
online = card_filter_data[card_filter_data['large_category'] == '온라인쇼핑']

In [ ]:
# offline의 pay_date 타입 변환하기
offline['pay_date'] = pd.to_datetime(offline['pay_date'])

In [ ]:
# 오프라인 거래 데이터에서 고객별로 최근 방문일, 방문횟수, 구매금액 기준으로 점수를 구하기
# tqdm은 for loop를 실행할 때 progress bar를 출력해주는 모듈, 이를 통해 남은 시간을 파악 가능
from tqdm import tqdm

# 중복 제거된 고객 아이디
customer_id = list(offline['person_id'].unique())

# 각 고객별로 구매금액 얼마인지 파악하기
monetary_df = pd.DataFrame() # 빈 데이터 프레임 생성
monetary_df['person_id'] = customer_id # 고객아이디 삽입

monetary_data = [] # 구매금액 담을 리스트

In [ ]:
# 고객별 총 구매금액 넣기
for ci in tqdm(customer_id):
    temp = offline.query('person_id == @ci') # 해당 아이디의 고객 데이터 추출
    amount = sum(temp['amount'])
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data

In [ ]:
# recency 각 고객별 최근방문일 알아보기
temp_recency_df = offline[['person_id', 'pay_date']].drop_duplicates()
recency_df = temp_recency_df.groupby('person_id')['pay_date'].max().reset_index()
recency_df = recency_df.rename(columns = {'pay_date': 'Recency'})

In [ ]:
# Frequency 각 고객별 방문횟수를 알아보기
temp_frequency_df = offline[['person_id', 'pay_date']].drop_duplicates()
frequency_df = temp_frequency_df.groupby('person_id')['pay_date'].nunique().reset_index()
frequency_df = frequency_df.rename(columns = {'paydate' : 'Frequency'})

In [ ]:
# monetary_df 칼럼명 변경하기
monetary_df = monetary_df.rename(columns = {'Customer_ID' : 'person_id'})

In [ ]:
# 각 df 병합하기
rfm_df = pd.merge(recency_df, frequency_df, how = 'left', on = 'person_id')
rfm_df = pd.merge(rfm_df, monetary_df, how = 'left', on = 'person_id')

In [ ]:
# F, M은 정수형태라 점수를 매기기 쉽지만, R는 날짜 형식이라 점수를 매기는데 무리가 있다. 따라서 추가적인 작업 필요
# 1단계: 기준시간 설정 / 2단계: 시간차이 계산 / 3단계 초단위로 변환
rfm_df.info()

In [ ]:
rfm_df

In [ ]:
# 기준시간은 어떻게 잡을까?
# 시간의 크기는 크게 잡는다. 이유는 빈도, 가격 또한 클수록 큰 점수를 부여하기 때문에 동일하게 설정
# 2022-06-01로 설정 Recency의 min: 2022-07-01 max: 2023-06-30

current_day = pd.to_datetime('20220601')
time_diff = rfm_df['Recency']-current_day
time_in_seconds = [x.total_seconds() for x in time_diff] # total_seconds() 함수는 시간 차이를 초단위로 변환해준다.
rfm_df['Recency'] = time_in_seconds # 데이터 삽입하기

In [ ]:
rfm_df.to_csv("Desktop/rfm_df.csv")

In [ ]:
from tqdm import tqdm
rfm_df = pd.read_csv("Desktop/rfm_df.csv")

In [ ]:
rfm_df

In [ ]:
# online의 pay_date 타입 변환하기
online['pay_date'] = pd.to_datetime(online['pay_date'])

In [ ]:
# 온라인 거래 데이터에서 고객별로 최근 방문일, 방문횟수, 구매금액 기준으로 점수를 구하기
# tqdm은 for loop를 실행할 때 progress bar를 출력해주는 모듈, 이를 통해 남은 시간을 파악 가능
from tqdm import tqdm

# 중복 제거된 고객 아이디
customer_id = list(online['person_id'].unique())

# 각 고객별로 구매금액 얼마인지 파악하기
monetary_df = pd.DataFrame() # 빈 데이터 프레임 생성
monetary_df['person_id'] = customer_id # 고객아이디 삽입

monetary_data = [] # 구매금액 담을 리스트

In [ ]:
# 고객별 총 구매금액 넣기
for ci in tqdm(customer_id):
    temp = online.query('person_id == @ci') # 해당 아이디의 고객 데이터 추출
    amount = sum(temp['amount'])
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data

In [ ]:
# recency 각 고객별 최근방문일 알아보기
temp_recency_df = online[['person_id', 'pay_date']].drop_duplicates()
recency_df = temp_recency_df.groupby('person_id')['pay_date'].max().reset_index()
recency_df = recency_df.rename(columns = {'pay_date': 'Recency'})

In [ ]:
# Frequency 각 고객별 방문횟수를 알아보기
temp_frequency_df = online[['person_id', 'pay_date']].drop_duplicates()
frequency_df = temp_frequency_df.groupby('person_id')['pay_date'].nunique().reset_index()
frequency_df = frequency_df.rename(columns = {'paydate' : 'Frequency'})

In [ ]:
# 각 df 병합하기
rfm_df = pd.merge(recency_df, frequency_df, how = 'left', on = 'person_id')
rfm_df = pd.merge(rfm_df, monetary_df, how = 'left', on = 'person_id')

In [ ]:
# Recency 변환하기
current_date = pd.to_datetime("20230701")
time_diff = current_date - rfm_df['Recency']
time_diff_days = [x.days for x in time_diff]
rfm_df['Recency'] = time_diff_days

In [ ]:
# 성별, 나이 추가하기
info_df = online[['person_id', 'age_group', 'person_gender']].drop_duplicates()

In [ ]:
rfm_online = pd.merge(rfm_df, info_df, how = 'left', on = 'person_id')

In [ ]:
rfm_online.to_csv("Desktop/rfm_online.csv")

### RFM_offline 데이터 추가하기
> 기존 RFM 값에서 성별, 연령 추가

In [ ]:
rfm_offline = pd.read_csv("Desktop/rfm_offline.csv")
rfm_offline.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
info_off = offline[['person_id', 'age_group', 'person_gender']].drop_duplicates()

In [ ]:
rfm_offline = pd.merge(rfm_offline, info_off, how = 'left', on = 'person_id')

In [ ]:
rfm_offline.to_csv("Desktop/rfm_offline.csv")

In [ ]:
rfm_offline

### 편의점 RFM 고객 분석

In [ ]:
# pay_date 데이터 타입 변환하기
convenience_df['pay_date'] = pd.to_datetime(convenience_df['pay_date'])

In [ ]:
# 중복 제거된 고객 아이디
customer_id = list(convenience_df['person_id'].unique())

# 고객별 구매금액 얼마인지 파악하기
monetary_df = pd.DataFrame()
monetary_df['person_id'] = customer_id

monetary_data = [] # 구매금액 담을 리스트

In [ ]:
# 고객별 총 구매금액 구하기
for ci in tqdm(customer_id):
    temp = convenience_df.query('person_id == @ci')
    amount = sum(temp['amount'])
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data

In [ ]:
# recency 각 고객별 최근방문일 알아보기
temp_recency_df = convenience_df[['person_id', 'pay_date']].drop_duplicates()
recency_df = temp_recency_df.groupby('person_id')['pay_date'].max().reset_index()
recency_df = recency_df.rename(columns = {'pay_date' : 'Recency'})

In [ ]:
# Frequency 각 고객별 방문횟수 알아보기
temp_frequency_df = convenience_df[['person_id', 'pay_date']].drop_duplicates()
frequency_df = temp_frequency_df.groupby('person_id')['pay_date'].nunique().reset_index()
frequency_df = frequency_df.rename(columns = {'pay_date' : 'Frequency'})

In [ ]:
# 각 df 병합하기
convenience_rfm = pd.merge(recency_df, frequency_df, how = 'left', on = 'person_id')
convenience_rfm = pd.merge(convenience_rfm, monetary_df, how = 'left', on = 'person_id')

In [ ]:
# Recency day로 변환하기

# 최근 날짜 설정
current_date = pd.to_datetime('20230701')

# 날짜 차이 구하기
time_diff = current_date - convenience_rfm['Recency']

# 날짜 day로 변환
time_in_days = [x.days for x in time_diff]

# 변환한 날짜로 교체하기
convenience_rfm['Recency'] = time_in_days

In [ ]:
# 성별 / 나이 / 편의점 추가하기
info_df = convenience_df[['person_id', 'age_group', 'person_gender']].drop_duplicates()

convenience_rfm = pd.merge(convenience_rfm, info_df, how = 'left', on = 'person_id')

In [ ]:
convenience_rfm['category'] = '편의점'

In [ ]:
convenience_rfm.to_csv("Desktop/convenience_rfm.csv")

### 대형마트 RFM 고객 분석

In [ ]:
# pay_date 데이터 타입 변환하기
bigmart_df['pay_date'] = pd.to_datetime(bigmart_df['pay_date'])

In [ ]:
# 중복 제거된 고객 아이디
customer_id = list(bigmart_df['person_id'].unique())

# 고객별 구매금액 얼마인지 파악하기
monetary_df = pd.DataFrame()
monetary_df['person_id'] = customer_id

monetary_data = [] # 구매금액 담을 리스트

In [ ]:
# 고객별 총 구매금액 구하기
for ci in tqdm(customer_id):
    temp = bigmart_df.query('person_id == @ci')
    amount = sum(temp['amount'])
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data

In [ ]:
# recency 각 고객별 최근방문일 알아보기
temp_recency_df = bigmart_df[['person_id', 'pay_date']].drop_duplicates()
recency_df = temp_recency_df.groupby('person_id')['pay_date'].max().reset_index()
recency_df = recency_df.rename(columns = {'pay_date' : 'Recency'})

In [ ]:
# Frequency 각 고객별 방문횟수 알아보기
temp_frequency_df = bigmart_df[['person_id', 'pay_date']].drop_duplicates()
frequency_df = temp_frequency_df.groupby('person_id')['pay_date'].nunique().reset_index()
frequency_df = frequency_df.rename(columns = {'pay_date' : 'Frequency'})

In [ ]:
# 각 df 병합하기
bigmart_rfm = pd.merge(recency_df, frequency_df, how = 'left', on = 'person_id')
bigmart_rfm = pd.merge(bigmart_rfm, monetary_df, how = 'left', on = 'person_id')

In [ ]:
# Recency day로 변환하기

# 최근 날짜 설정
current_date = pd.to_datetime('20230701')

# 날짜 차이 구하기
time_diff = current_date - bigmart_rfm['Recency']

# 날짜 day로 변환
time_in_days = [x.days for x in time_diff]

# 변환한 날짜로 교체하기
bigmart_rfm['Recency'] = time_in_days

In [ ]:
# 성별 / 나이 / 편의점 추가하기
info_df = bigmart_df[['person_id', 'age_group', 'person_gender']].drop_duplicates()

bigmart_rfm = pd.merge(bigmart_rfm, info_df, how = 'left', on = 'person_id')

bigmart_rfm['category'] = '대형마트'

In [ ]:
bigmart_rfm.to_csv("Desktop/bigmart_rfm.csv")

### 마트 RFM 고객 분석

In [ ]:
# pay_date 데이터 타입 변환하기
smallmart_df['pay_date'] = pd.to_datetime(smallmart_df['pay_date'])

In [ ]:
# 중복 제거된 고객 아이디
customer_id = list(smallmart_df['person_id'].unique())

# 고객별 구매금액 얼마인지 파악하기
monetary_df = pd.DataFrame()
monetary_df['person_id'] = customer_id

monetary_data = [] # 구매금액 담을 리스트

In [ ]:
# 고객별 총 구매금액 구하기
for ci in tqdm(customer_id):
    temp = smallmart_df.query('person_id == @ci')
    amount = sum(temp['amount'])
    monetary_data.append(amount)
    
monetary_df['Monetary'] = monetary_data

In [ ]:
# recency 각 고객별 최근방문일 알아보기
temp_recency_df = smallmart_df[['person_id', 'pay_date']].drop_duplicates()
recency_df = temp_recency_df.groupby('person_id')['pay_date'].max().reset_index()
recency_df = recency_df.rename(columns = {'pay_date' : 'Recency'})

In [ ]:
# Frequency 각 고객별 방문횟수 알아보기
temp_frequency_df = smallmart_df[['person_id', 'pay_date']].drop_duplicates()
frequency_df = temp_frequency_df.groupby('person_id')['pay_date'].nunique().reset_index()
frequency_df = frequency_df.rename(columns = {'pay_date' : 'Frequency'})

In [ ]:
# 각 df 병합하기
smallmart_rfm = pd.merge(recency_df, frequency_df, how = 'left', on = 'person_id')
smallmart_rfm = pd.merge(smallmart_rfm, monetary_df, how = 'left', on = 'person_id')

In [ ]:
# Recency day로 변환하기

# 최근 날짜 설정
current_date = pd.to_datetime('20230701')

# 날짜 차이 구하기
time_diff = current_date - smallmart_rfm['Recency']

# 날짜 day로 변환
time_in_days = [x.days for x in time_diff]

# 변환한 날짜로 교체하기
smallmart_rfm['Recency'] = time_in_days

In [ ]:
# 성별 / 나이 / 편의점 추가하기
info_df = smallmart_df[['person_id', 'age_group', 'person_gender']].drop_duplicates()

smallmart_rfm = pd.merge(smallmart_rfm, info_df, how = 'left', on = 'person_id')

smallmart_rfm['category'] = '슈퍼마켓'

In [ ]:
smallmart_rfm.to_csv("Desktop/smallmart_rfm.csv")

### RFM 각 변수에 대한 점수별 기준 산정
참고자료: https://stelch.tistory.com/pages/CRM-RFM-%EA%B3%A0%EA%B0%9D%EA%B0%80%EC%B9%98%EB%B6%84%EC%84%9D-feat-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%99%9C%EC%9A%A9

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from datetime import datetime
from tqdm import tqdm

# 데이터 불러오기
c_rfm = pd.read_csv("Desktop/convenience_rfm.csv")
b_rfm = pd.read_csv("Desktop/bigmart_rfm.csv")
s_rfm = pd.read_csv("Desktop/smallmart_rfm.csv")

In [ ]:
# 편의점 최근 구매 날짜
# 평균적으로 고객 한 명당 최근에 구매한 날짜는 109일 전
c_rfm.Recency.describe()

In [ ]:
# 시간 차이의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(c_rfm['Recency'], bins = 30)
plt.show()

# 최근 결제 수가 많다는 걸 알 수 있다.

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_recency = c_rfm[['Recency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_recency)
    df_recency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.plot(list(point.keys()), list(point.values()))
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(c_rfm[['Recency']])
c_rfm['Recencycluster'] = kmeans.predict(c_rfm[['Recency']])

# cluster 분석 모형
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by = target_field_name, ascending = ascending).reset_index(drop = True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, 'index']], on = cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis = 1)
    df_final = df_final.rename(columns = {"index" : cluster_field_name})
    return df_final

c_rfm = order_cluster('Recencycluster', 'Recency', c_rfm, False)
c_rfm.groupby('Recencycluster')['Recency'].describe()

In [ ]:
# Frequency 도출을 위한 데이터 추출
c_rfm.Frequency.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'whitegrid')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(c_rfm['Frequency'], bins = 30)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_frequency = c_rfm[['Frequency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_frequency)
    df_frequency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(c_rfm[['Frequency']])
c_rfm['Frequencycluster'] = kmeans.predict(c_rfm[['Frequency']])

c_rfm = order_cluster('Frequencycluster', 'Frequency', c_rfm, True)
c_rfm.groupby('Frequencycluster')['Frequency'].describe()

In [ ]:
# Monetary 도출을 위한 데이터 추출
c_rfm.Monetary.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(c_rfm['Monetary'], hist = False)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_monetary = c_rfm[['Monetary']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_monetary)
    df_monetary['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(c_rfm[['Monetary']])
c_rfm['Monetarycluster'] = kmeans.predict(c_rfm[['Monetary']])

c_rfm = order_cluster('Monetarycluster', 'Monetary', c_rfm, True)
c_rfm.groupby('Monetarycluster')['Monetary'].describe()

In [ ]:
c_rfm.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
c_rfm.to_csv("Desktop/convenience_rfm.csv")

### 대형마트 RFM 점수 산정

In [ ]:
# 대형마트 최근 구매 날짜
# 평균적으로 고객 한 명당 최근에 구매한 날짜는 123일 전
b_rfm.Recency.describe()

In [ ]:
# 시간 차이의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(b_rfm['Recency'], bins = 30)
plt.show()

# 최근 결제 수가 많다는 걸 알 수 있다.

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_recency = b_rfm[['Recency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_recency)
    df_recency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.plot(list(point.keys()), list(point.values()))
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(b_rfm[['Recency']])
b_rfm['Recencycluster'] = kmeans.predict(b_rfm[['Recency']])

b_rfm = order_cluster('Recencycluster', 'Recency', b_rfm, False)
b_rfm.groupby('Recencycluster')['Recency'].describe()

In [ ]:
# Frequency 도출을 위한 데이터 추출
b_rfm.Frequency.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'whitegrid')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(b_rfm['Frequency'], bins = 30)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_frequency = b_rfm[['Frequency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_frequency)
    df_frequency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(b_rfm[['Frequency']])
b_rfm['Frequencycluster'] = kmeans.predict(b_rfm[['Frequency']])

b_rfm = order_cluster('Frequencycluster', 'Frequency', b_rfm, True)
b_rfm.groupby('Frequencycluster')['Frequency'].describe()

In [ ]:
# Monetary 도출을 위한 데이터 추출
b_rfm.Monetary.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(b_rfm['Monetary'], hist = False)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_monetary = b_rfm[['Monetary']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_monetary)
    df_monetary['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(b_rfm[['Monetary']])
b_rfm['Monetarycluster'] = kmeans.predict(b_rfm[['Monetary']])

b_rfm = order_cluster('Monetarycluster', 'Monetary', b_rfm, True)
b_rfm.groupby('Monetarycluster')['Monetary'].describe()

In [ ]:
b_rfm.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
b_rfm.to_csv("Desktop/bigmart_rfm.csv")

### 슈퍼마켓 RFM 점수 산정

In [ ]:
# 편의점 최근 구매 날짜
# 평균적으로 고객 한 명당 최근에 구매한 날짜는 125일 전
s_rfm.Recency.describe()

In [ ]:
# 시간 차이의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(s_rfm['Recency'], bins = 30)
plt.show()

# 최근 결제 수가 많다는 걸 알 수 있다.

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_recency = s_rfm[['Recency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_recency)
    df_recency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.plot(list(point.keys()), list(point.values()))
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(s_rfm[['Recency']])
s_rfm['Recencycluster'] = kmeans.predict(s_rfm[['Recency']])


s_rfm = order_cluster('Recencycluster', 'Recency', s_rfm, False)
s_rfm.groupby('Recencycluster')['Recency'].describe()

In [ ]:
# Frequency 도출을 위한 데이터 추출
s_rfm.Frequency.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'whitegrid')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(s_rfm['Frequency'], bins = 30)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_frequency = s_rfm[['Frequency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_frequency)
    df_frequency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(s_rfm[['Frequency']])
s_rfm['Frequencycluster'] = kmeans.predict(s_rfm[['Frequency']])

s_rfm = order_cluster('Frequencycluster', 'Frequency', s_rfm, True)
s_rfm.groupby('Frequencycluster')['Frequency'].describe()

In [ ]:
# Monetary 도출을 위한 데이터 추출
c_rfm.Monetary.describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(s_rfm['Monetary'], hist = False)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_monetary = s_rfm[['Monetary']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_monetary)
    df_monetary['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(s_rfm[['Monetary']])
s_rfm['Monetarycluster'] = kmeans.predict(s_rfm[['Monetary']])

s_rfm = order_cluster('Monetarycluster', 'Monetary', s_rfm, True)
s_rfm.groupby('Monetarycluster')['Monetary'].describe()

### 편의점 / 대형마트 / 슈퍼마켓의 분산분석 결과
> 모든 평균의 차이는 통계적으로 유의미하다는 결과를 얻음

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


large_df = np.array(list(df_1['Recency']))
regular_df = np.array(list(df_2['Recency']))
convenience_df = np.array(list(df['Recency']))

data = {
    "group" : ["big"] * len(large_df) + ["small"] * len(regular_df) + ["convenience"] * len(convenience_df),
    "Recency" : np.concatenate([large_df, regular_df, convenience_df]) 
}

df = pd.DataFrame(data)

In [ ]:
from scipy.stats import f_oneway

f_statistic, p_value = f_oneway(large_df, regular_df, convenience)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05

if p_value < alpha:
    print("그룹 간의 평균 차이는 통계적으로 유의미합니다.")
    
else:
    print("그룹 간의 평균 차이는 통계적으로 유의미하지 않습니다.")

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x = "group", y = "Recency", data = df)
plt.xlabel("group")
plt.ylabel("Recency")
plt.title("result")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

large_df = np.array(list(df_1['Frequency']))
regular_df = np.array(list(df_2['Frequency']))
convenience_df = np.array(list(df['Frequency']))

data = {
    "group" : ["big"] * len(large_df) + ["small"] * len(regular_df) + ["convenience"] * len(convenience_df),
    "Frequency" : np.concatenate([large_df, regular_df, convenience_df]) 
}

df = pd.DataFrame(data)

In [ ]:
from scipy.stats import f_oneway

f_statistic, p_value = f_oneway(large_df, regular_df, convenience)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05

if p_value < alpha:
    print("그룹 간의 평균 차이는 통계적으로 유의미합니다.")
    
else:
    print("그룹 간의 평균 차이는 통계적으로 유의미하지 않습니다.")

In [ ]:
plt.figure(figsize = (10, 6))
sns.boxplot(x = "group", y = "Frequency", data = df)
plt.xlabel("group")
plt.ylabel("Frequency")
plt.title("result")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

large_df = np.array(list(df_1['Monetary']))
regular_df = np.array(list(df_2['Monetary']))
convenience_df = np.array(list(df['Monetary']))

data = {
    "group" : ["big"] * len(large_df) + ["small"] * len(regular_df) + ["convenience"] * len(convenience_df),
    "Monetary" : np.concatenate([large_df, regular_df, convenience_df]) 
}

df = pd.DataFrame(data)

In [ ]:
from scipy.stats import f_oneway

f_statistic, p_value = f_oneway(large_df, regular_df, convenience)

print("F-statistic:", f_statistic)
print("p-value:", p_value)

alpha = 0.05

if p_value < alpha:
    print("그룹 간의 평균 차이는 통계적으로 유의미합니다.")
    
else:
    print("그룹 간의 평균 차이는 통계적으로 유의미하지 않습니다.")

In [ ]:
plt.figure(figsize = (10, 6))
sns.pointplot(x = "group", y = "Monetary", data = df, ci = "sd")
plt.xlabel("group")
plt.ylabel("Frequency")
plt.title("result")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from datetime import datetime
from tqdm import tqdm

df = pd.read_csv("Desktop/rfm_online.csv")

In [ ]:
# 시간 차이의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(df['Recency'], bins = 30)
plt.show()

# 최근 결제 수가 많다는 걸 알 수 있다.

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_recency = df[['Recency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_recency)
    df_recency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.plot(list(point.keys()), list(point.values()))
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(df[['Recency']])
df['Recencycluster'] = kmeans.predict(df[['Recency']])

# cluster 분석 모형
def order_cluster(cluster_field_name, target_field_name, df, ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by = target_field_name, ascending = ascending).reset_index(drop = True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df, df_new[[cluster_field_name, 'index']], on = cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis = 1)
    df_final = df_final.rename(columns = {"index" : cluster_field_name})
    return df_final

df = order_cluster('Recencycluster', 'Recency', df, False)
df.groupby('Recencycluster')['Recency'].describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'whitegrid')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(df['Frequency'], bins = 30)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_frequency = df[['Frequency']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_frequency)
    df_frequency['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(df[['Frequency']])
df['Frequencycluster'] = kmeans.predict(df[['Frequency']])

df = order_cluster('Frequencycluster', 'Frequency', df, True)
df.groupby('Frequencycluster')['Frequency'].describe()

In [ ]:
# 구매 빈도의 흐름 시각화
sns.set(palette = 'muted', color_codes = True, style = 'white')
fig, ax = plt.subplots(figsize = (12, 6))
sns.despine(left = True)
sns.distplot(df['Monetary'], hist = False)
plt.show()

In [ ]:
# kmeans 계산하기 위한 elbow point구하기
from sklearn.cluster import KMeans

point = {}
df_monetary = df[['Monetary']]

for k in range(1, 10):
    kmeans = KMeans(n_clusters = k, max_iter = 100).fit(df_monetary)
    df_monetary['cluster'] = kmeans.labels_
    point[k] = kmeans.inertia_

plt.figure(figsize = (10, 5))
plt.plot(list(point.keys()), list(point.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
# 위 그래프를 통해서 elbow point는 4인 지점으로 설정
kmeans = KMeans(n_clusters = 4)
kmeans.fit(df[['Monetary']])
df['Monetarycluster'] = kmeans.predict(df[['Monetary']])

df = order_cluster('Monetarycluster', 'Monetary', df, True)
df.groupby('Monetarycluster')['Monetary'].describe()

In [ ]:
import pandas as pd

rfm = pd.read_csv("Desktop/RFM 통합.csv", index_col = 0)

In [ ]:
rfm[rfm['category'] == '대형마트']